In [2]:
%pip install --upgrade pip
%pip install --upgrade transformers
%pip install --upgrade torch
%pip install --upgrade pandas
# %pip uninstall TensorFlow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Data Processing

In [9]:
import pandas as pd
pd.set_option('display.max_colwidth',0)
df = pd.read_csv('topic_data_first_sen.csv')
df = df.drop("List of Sentences", axis=1)
df.head(5)

,Topic,Term,Term Percentage,first_sentence
0,Topic0,women,0.010,"['In the last 20\xa0years, aspecific auto-immunity, defined as positivity of auto-antibodies in blood sample without clinical or biological criteria for defined diseases, has been evoked in a subpopulation of infertile women"
1,Topic0,pregnancy,0.008,"['The direct pathogenesis of this auto-immunity is unknown, but therapeutic immunomodulators, prescribed on a case-by-case basis, could favor pregnancy even in cases of unexplained primary or secondary infertility"
2,Topic0,risk,0.008,['Meta-analysis of studies that evaluated the role of NK cells in IVF outcome showed no significant difference in live birth rates in women with elevated NK cells or NK cell activity compared with women without elevated peripheral NK cells or NK cell activity (NK activity assessed using a cytotoxicity assay) (relative risk 0
3,Topic0,infertility,0.007,"['On average, 10\xa0% of infertile couples have unexplained infertility"
4,Topic0,review,0.007,"['A systematic review was performed (PUBMED) using the MESH search terms ""infertility"" and ""auto-immunity"" or ""reproductive technique"" or ""assisted reproduction"" or ""in vitro fertilization"" and ""auto-immunity"


In [10]:
import re
def clean_text(text):
    # Remove special characters, punctuation, and extra whitespace
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    # Remove non-breaking spaces (replace '\xa0' with a regular space)
    cleaned_text = cleaned_text.replace('\xa0', ' ')
    cleaned_text = cleaned_text.replace('xa0', ' ')
    # Remove extra whitespace and trim the text
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text
# Apply the clean_text function to your DataFrame
df['first_sentence'] = df['first_sentence'].apply(clean_text)
# Print the cleaned DataFrame
print(df['first_sentence'])


0     In the last 20 years aspecific auto immunity defined as positivity of auto antibodies in blood sample without clinical or biological criteria for defined diseases has been evoked in a subpopulation of infertile women                                                                                                         
1     The direct pathogenesis of this auto immunity is unknown but therapeutic immunomodulators prescribed on a case by case basis could favor pregnancy even in cases of unexplained primary or secondary infertility                                                                                                                 
2     Meta analysis of studies that evaluated the role of NK cells in IVF outcome showed no significant difference in live birth rates in women with elevated NK cells or NK cell activity compared with women without elevated peripheral NK cells or NK cell activity NK activity assessed using a cytotoxicity assay relative risk 0
3     On average

Some models with bio datasets

In [5]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

/Users/tggs636/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
def tokenModelAndPipe(model_name):
    # Load model directly
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(model_name)
    # Use a pipeline as a high-level helper
    pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
    df['ner_results'] = df['first_sentence'].apply(lambda sentence: pipe(sentence))
    

BERT

In [8]:

tokenModelAndPipe("parsi-ai-nlpclass/Gp3_medical_token_classification")
df[['first_sentence', 'ner_results']].head(5)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,first_sentence,ner_results
0,In the last 20 years aspecific auto immunity defined as positivity of auto antibodies in blood sample without clinical or biological criteria for defined diseases has been evoked in a subpopulation of infertile women,[]
1,The direct pathogenesis of this auto immunity is unknown but therapeutic immunomodulators prescribed on a case by case basis could favor pregnancy even in cases of unexplained primary or secondary infertility,"[{'entity_group': 'Disease', 'score': 0.98479956, 'word': 'infertility', 'start': 197, 'end': 208}]"
2,Meta analysis of studies that evaluated the role of NK cells in IVF outcome showed no significant difference in live birth rates in women with elevated NK cells or NK cell activity compared with women without elevated peripheral NK cells or NK cell activity NK activity assessed using a cytotoxicity assay relative risk 0,"[{'entity_group': 'Disease', 'score': 0.7056203, 'word': '##f', 'start': 66, 'end': 67}]"
3,On average 10 of infertile couples have unexplained infertility,"[{'entity_group': 'Disease', 'score': 0.85665673, 'word': 'infertile', 'start': 17, 'end': 26}, {'entity_group': 'Disease', 'score': 0.997083, 'word': 'infertility', 'start': 52, 'end': 63}]"
4,A systematic review was performed PUBMED using the MESH search terms infertility and auto immunity or reproductive technique or assisted reproduction or in vitro fertilization and auto immunity,"[{'entity_group': 'Disease', 'score': 0.99741316, 'word': 'infertility', 'start': 69, 'end': 80}]"


In [91]:
tokenModelAndPipe("samrawal/bert-base-uncased_clinical-ner")
df[['first_sentence', 'ner_results']].head(5)

,first_sentence,ner_results
0,In the last 20 years aspecific auto immunity defined as positivity of auto antibodies in blood sample without clinical or biological criteria for defined diseases has been evoked in a subpopulation of infertile women,"[{'entity_group': 'problem', 'score': 0.954476, 'word': 'aspecifi', 'start': 21, 'end': 29}, {'entity_group': 'problem', 'score': 0.98202425, 'word': '##c auto immunity', 'start': 29, 'end': 44}, {'entity_group': 'problem', 'score': 0.6493882, 'word': 'posit', 'start': 56, 'end': 61}, {'entity_group': 'treatment', 'score': 0.4956661, 'word': 'auto antibodies', 'start': 70, 'end': 85}, {'entity_group': 'test', 'score': 0.9123425, 'word': 'blood sample', 'start': 89, 'end': 101}, {'entity_group': 'problem', 'score': 0.9788437, 'word': 'defined diseases', 'start': 146, 'end': 162}]"
1,The direct pathogenesis of this auto immunity is unknown but therapeutic immunomodulators prescribed on a case by case basis could favor pregnancy even in cases of unexplained primary or secondary infertility,"[{'entity_group': 'problem', 'score': 0.9888324, 'word': 'this auto immunity', 'start': 27, 'end': 45}, {'entity_group': 'treatment', 'score': 0.99035275, 'word': 'therapeutic immunomodulators', 'start': 61, 'end': 89}, {'entity_group': 'problem', 'score': 0.7743186, 'word': 'pregnancy', 'start': 137, 'end': 146}, {'entity_group': 'problem', 'score': 0.9490906, 'word': 'unexplained primary or secondary infertility', 'start': 164, 'end': 208}]"
2,Meta analysis of studies that evaluated the role of NK cells in IVF outcome showed no significant difference in live birth rates in women with elevated NK cells or NK cell activity compared with women without elevated peripheral NK cells or NK cell activity NK activity assessed using a cytotoxicity assay relative risk 0,"[{'entity_group': 'test', 'score': 0.8733111, 'word': 'meta analysis', 'start': 0, 'end': 13}, {'entity_group': 'treatment', 'score': 0.66923594, 'word': 'nk cells', 'start': 52, 'end': 60}, {'entity_group': 'treatment', 'score': 0.5524435, 'word': 'ivf', 'start': 64, 'end': 67}, {'entity_group': 'test', 'score': 0.8369388, 'word': 'rates', 'start': 123, 'end': 128}, {'entity_group': 'problem', 'score': 0.9981602, 'word': 'elevated nk cells', 'start': 143, 'end': 160}, {'entity_group': 'problem', 'score': 0.958132, 'word': 'nk cell activity', 'start': 164, 'end': 180}, {'entity_group': 'problem', 'score': 0.99847466, 'word': 'elevated peripheral nk cells', 'start': 209, 'end': 237}, {'entity_group': 'problem', 'score': 0.9935451, 'word': 'nk cell activity', 'start': 241, 'end': 257}, {'entity_group': 'test', 'score': 0.9815953, 'word': 'nk activity', 'start': 258, 'end': 269}, {'entity_group': 'test', 'score': 0.89992523, 'word': 'a cytotoxicity assay', 'start': 285, 'end': 305}]"
3,On average 10 of infertile couples have unexplained infertility,"[{'entity_group': 'problem', 'score': 0.98499477, 'word': 'unexplained infertility', 'start': 40, 'end': 63}]"
4,A systematic review was performed PUBMED using the MESH search terms infertility and auto immunity or reproductive technique or assisted reproduction or in vitro fertilization and auto immunity,"[{'entity_group': 'problem', 'score': 0.8016464, 'word': 'infertility', 'start': 69, 'end': 80}, {'entity_group': 'problem', 'score': 0.44788632, 'word': 'auto', 'start': 85, 'end': 89}, {'entity_group': 'treatment', 'score': 0.5271289, 'word': 'immunity', 'start': 90, 'end': 98}, {'entity_group': 'treatment', 'score': 0.5443014, 'word': 'reproductive technique', 'start': 102, 'end': 124}, {'entity_group': 'treatment', 'score': 0.50382745, 'word': 'reproduction', 'start': 137, 'end': 149}, {'entity_group': 'treatment', 'score': 0.8218632, 'word': 'in vitro fertilization', 'start': 153, 'end': 175}, {'entity_group': 'treatment', 'score': 0.831735, 'word': 'auto immunity', 'start': 180, 'end': 193}]"


In [92]:
tokenModelAndPipe("pruas/BENT-PubMedBERT-NER-Disease")
df[['first_sentence', 'ner_results']].head(5)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,first_sentence,ner_results
0,In the last 20 years aspecific auto immunity defined as positivity of auto antibodies in blood sample without clinical or biological criteria for defined diseases has been evoked in a subpopulation of infertile women,[]
1,The direct pathogenesis of this auto immunity is unknown but therapeutic immunomodulators prescribed on a case by case basis could favor pregnancy even in cases of unexplained primary or secondary infertility,"[{'entity_group': 'I', 'score': 0.47189173, 'word': 'infertility', 'start': 197, 'end': 208}]"
2,Meta analysis of studies that evaluated the role of NK cells in IVF outcome showed no significant difference in live birth rates in women with elevated NK cells or NK cell activity compared with women without elevated peripheral NK cells or NK cell activity NK activity assessed using a cytotoxicity assay relative risk 0,[]
3,On average 10 of infertile couples have unexplained infertility,"[{'entity_group': 'B', 'score': 0.67905885, 'word': 'unexplained', 'start': 40, 'end': 51}, {'entity_group': 'I', 'score': 0.7268239, 'word': 'infertility', 'start': 52, 'end': 63}]"
4,A systematic review was performed PUBMED using the MESH search terms infertility and auto immunity or reproductive technique or assisted reproduction or in vitro fertilization and auto immunity,[]


In [93]:
tokenModelAndPipe("medical-ner-proj/bert-medical-ner-proj")
df[['first_sentence', 'ner_results']].head(5)

,first_sentence,ner_results
0,In the last 20 years aspecific auto immunity defined as positivity of auto antibodies in blood sample without clinical or biological criteria for defined diseases has been evoked in a subpopulation of infertile women,"[{'entity_group': 'B_problem', 'score': 0.9255191, 'word': 'as', 'start': 21, 'end': 23}, {'entity_group': 'I_problem', 'score': 0.9130235, 'word': '##pecific auto immunity', 'start': 23, 'end': 44}, {'entity_group': 'B_problem', 'score': 0.95112675, 'word': 'po', 'start': 56, 'end': 58}, {'entity_group': 'I_problem', 'score': 0.90929115, 'word': '##sit', 'start': 58, 'end': 61}, {'entity_group': 'B_problem', 'score': 0.560097, 'word': '##ivity', 'start': 61, 'end': 66}, {'entity_group': 'I_problem', 'score': 0.47221294, 'word': 'of auto', 'start': 67, 'end': 74}, {'entity_group': 'I_test', 'score': 0.5714571, 'word': 'antibodies', 'start': 75, 'end': 85}, {'entity_group': 'B_test', 'score': 0.9674645, 'word': 'blood', 'start': 89, 'end': 94}, {'entity_group': 'I_test', 'score': 0.93470395, 'word': 'sample', 'start': 95, 'end': 101}, {'entity_group': 'B_problem', 'score': 0.7074107, 'word': 'defined', 'start': 146, 'end': 153}, {'entity_group': 'I_problem', 'score': 0.9637322, 'word': 'diseases', 'start': 154, 'end': 162}]"
1,The direct pathogenesis of this auto immunity is unknown but therapeutic immunomodulators prescribed on a case by case basis could favor pregnancy even in cases of unexplained primary or secondary infertility,"[{'entity_group': 'B_problem', 'score': 0.6604742, 'word': 'this', 'start': 27, 'end': 31}, {'entity_group': 'I_problem', 'score': 0.9514333, 'word': 'auto immunity', 'start': 32, 'end': 45}, {'entity_group': 'B_treatment', 'score': 0.9917883, 'word': 'therapeutic', 'start': 61, 'end': 72}, {'entity_group': 'I_treatment', 'score': 0.9518787, 'word': 'immunomodulators', 'start': 73, 'end': 89}, {'entity_group': 'B_problem', 'score': 0.9934348, 'word': 'une', 'start': 164, 'end': 167}, {'entity_group': 'I_problem', 'score': 0.9902399, 'word': '##xplained primary or secondary infertility', 'start': 167, 'end': 208}]"
2,Meta analysis of studies that evaluated the role of NK cells in IVF outcome showed no significant difference in live birth rates in women with elevated NK cells or NK cell activity compared with women without elevated peripheral NK cells or NK cell activity NK activity assessed using a cytotoxicity assay relative risk 0,"[{'entity_group': 'B_test', 'score': 0.95373404, 'word': 'meta', 'start': 0, 'end': 4}, {'entity_group': 'I_test', 'score': 0.9820785, 'word': 'analysis', 'start': 5, 'end': 13}, {'entity_group': 'B_test', 'score': 0.5714108, 'word': 'studies', 'start': 17, 'end': 24}, {'entity_group': 'B_pronoun', 'score': 0.9974642, 'word': 'that', 'start': 25, 'end': 29}, {'entity_group': 'B_test', 'score': 0.67591494, 'word': 'nk', 'start': 52, 'end': 54}, {'entity_group': 'I_test', 'score': 0.53159314, 'word': 'cells', 'start': 55, 'end': 60}, {'entity_group': 'B_treatment', 'score': 0.5132925, 'word': 'iv', 'start': 64, 'end': 66}, {'entity_group': 'I_treatment', 'score': 0.33561403, 'word': '##f', 'start': 66, 'end': 67}, {'entity_group': 'B_test', 'score': 0.9181432, 'word': 'live', 'start': 112, 'end': 116}, {'entity_group': 'I_test', 'score': 0.8977128, 'word': 'birth rates', 'start': 117, 'end': 128}, {'entity_group': 'B_problem', 'score': 0.9938216, 'word': 'elevated', 'start': 143, 'end': 151}, {'entity_group': 'I_problem', 'score': 0.9374776, 'word': 'nk cells or nk cell activity', 'start': 152, 'end': 180}, {'entity_group': 'B_problem', 'score': 0.996003, 'word': 'elevated', 'start': 209, 'end': 217}, {'entity_group': 'I_problem', 'score': 0.9266063, 'word': 'peripheral nk cells or nk cell activity', 'start': 218, 'end': 257}, {'entity_group': 'B_test', 'score': 0.96559507, 'word': 'nk', 'start': 258, 'end': 260}, {'entity_group': 'I_test', 'score': 0.9640924, 'word': 'activity', 'start': 261, 'end': 269}, {'entity_group': 'B_tes

ELECTRA

In [94]:
tokenModelAndPipe("kamalkraj/BioELECTRA-PICO")
df[['first_sentence', 'ner_results']].head(5)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,first_sentence,ner_results
0,In the last 20 years aspecific auto immunity defined as positivity of auto antibodies in blood sample without clinical or biological criteria for defined diseases has been evoked in a subpopulation of infertile women,"[{'entity_group': 'Participants', 'score': 0.9600016, 'word': 'infertile women', 'start': 201, 'end': 216}]"
1,The direct pathogenesis of this auto immunity is unknown but therapeutic immunomodulators prescribed on a case by case basis could favor pregnancy even in cases of unexplained primary or secondary infertility,"[{'entity_group': 'Intervention', 'score': 0.505901, 'word': 'immunomod', 'start': 73, 'end': 82}]"
2,Meta analysis of studies that evaluated the role of NK cells in IVF outcome showed no significant difference in live birth rates in women with elevated NK cells or NK cell activity compared with women without elevated peripheral NK cells or NK cell activity NK activity assessed using a cytotoxicity assay relative risk 0,"[{'entity_group': 'Outcome', 'score': 0.8684463, 'word': 'live birth rates', 'start': 112, 'end': 128}, {'entity_group': 'Participants', 'score': 0.58842725, 'word': 'women with elevated nk', 'start': 132, 'end': 154}, {'entity_group': 'Participants', 'score': 0.61475736, 'word': 'women', 'start': 195, 'end': 200}]"
3,On average 10 of infertile couples have unexplained infertility,"[{'entity_group': 'Participants', 'score': 0.70210934, 'word': 'infertile couples have unexplained infertility', 'start': 17, 'end': 63}]"
4,A systematic review was performed PUBMED using the MESH search terms infertility and auto immunity or reproductive technique or assisted reproduction or in vitro fertilization and auto immunity,[]


ALBERT

In [95]:
tokenModelAndPipe("medical-ner-proj/albert-medical-ner-proj")
df[['first_sentence', 'ner_results']].head(5)

,first_sentence,ner_results
0,In the last 20 years aspecific auto immunity defined as positivity of auto antibodies in blood sample without clinical or biological criteria for defined diseases has been evoked in a subpopulation of infertile women,"[{'entity_group': 'B_problem', 'score': 0.5197924, 'word': 'a', 'start': 21, 'end': 22}, {'entity_group': 'I_problem', 'score': 0.63372016, 'word': 'specific auto immunity', 'start': 22, 'end': 44}, {'entity_group': 'B_test', 'score': 0.77859074, 'word': 'auto', 'start': 70, 'end': 74}, {'entity_group': 'I_test', 'score': 0.841563, 'word': 'antibodies', 'start': 75, 'end': 85}, {'entity_group': 'B_test', 'score': 0.59789896, 'word': 'blood', 'start': 89, 'end': 94}, {'entity_group': 'I_test', 'score': 0.88789254, 'word': 'sample', 'start': 95, 'end': 101}, {'entity_group': 'B_problem', 'score': 0.9724062, 'word': 'defined', 'start': 146, 'end': 153}, {'entity_group': 'I_problem', 'score': 0.98557633, 'word': 'diseases', 'start': 154, 'end': 162}]"
1,The direct pathogenesis of this auto immunity is unknown but therapeutic immunomodulators prescribed on a case by case basis could favor pregnancy even in cases of unexplained primary or secondary infertility,"[{'entity_group': 'B_pronoun', 'score': 0.68769795, 'word': 'this', 'start': 27, 'end': 31}, {'entity_group': 'I_problem', 'score': 0.95559144, 'word': 'auto immunity', 'start': 32, 'end': 45}, {'entity_group': 'B_treatment', 'score': 0.94914347, 'word': '', 'start': 61, 'end': 62}, {'entity_group': 'I_treatment', 'score': 0.9062583, 'word': 'therapeutic immunomodulators', 'start': 61, 'end': 89}, {'entity_group': 'B_problem', 'score': 0.9518316, 'word': 'unexplained', 'start': 164, 'end': 175}, {'entity_group': 'I_problem', 'score': 0.9901222, 'word': 'primary or secondary infertility', 'start': 176, 'end': 208}]"
2,Meta analysis of studies that evaluated the role of NK cells in IVF outcome showed no significant difference in live birth rates in women with elevated NK cells or NK cell activity compared with women without elevated peripheral NK cells or NK cell activity NK activity assessed using a cytotoxicity assay relative risk 0,"[{'entity_group': 'B_test', 'score': 0.9876881, 'word': 'meta', 'start': 0, 'end': 4}, {'entity_group': 'I_test', 'score': 0.98847026, 'word': 'analysis of studies', 'start': 5, 'end': 24}, {'entity_group': 'B_pronoun', 'score': 0.99143714, 'word': 'that', 'start': 25, 'end': 29}, {'entity_group': 'B_test', 'score': 0.50776565, 'word': 'nk', 'start': 52, 'end': 54}, {'entity_group': 'I_test', 'score': 0.92462647, 'word': 'cells', 'start': 55, 'end': 60}, {'entity_group': 'B_test', 'score': 0.83089817, 'word': 'i', 'start': 64, 'end': 65}, {'entity_group': 'I_test', 'score': 0.40468043, 'word': 'vf', 'start': 65, 'end': 67}, {'entity_group': 'B_test', 'score': 0.7683117, 'word': 'live', 'start': 112, 'end': 116}, {'entity_group': 'I_test', 'score': 0.7538996, 'word': 'birth rates', 'start': 117, 'end': 128}, {'entity_group': 'B_problem', 'score': 0.99332255, 'word': 'elevated', 'start': 143, 'end': 151}, {'entity_group': 'I_problem', 'score': 0.9957476, 'word': 'nk cells', 'start': 152, 'end': 160}, {'entity_group': 'B_problem', 'score': 0.87633634, 'word': '', 'start': 164, 'end': 165}, {'entity_group': 'I_problem', 'score': 0.9748767, 'word': 'nk cell activity', 'start': 164, 'end': 180}, {'entity_group': 'B_problem', 'score': 0.99369025, 'word': 'elevated', 'start': 209, 'end': 217}, {'entity_group': 'I_problem', 'score': 0.993258, 'word': 'peripheral nk cells', 'start': 218, 'end': 237}, {'entity_group': 'B_problem', 'score': 0.67778707, 'word': '', 'start': 241, 'end': 242}, {'entity_group': 'I_problem', 'score': 0.9287834, 'word': 'nk cell activity', 'start': 241, 'end': 257}, {'entity_group': 'B_test', 'score': 0.5652019, 'word': '', 'start': 258, 'end': 259}, {'entity_group': 'I_test', 'score': 0.5760882, 'word': 'nk activity', 'start': 258, 'end': 269}, {'entity_group': 'B_test', 'score': 0.9855

roBERTa

In [96]:
tokenModelAndPipe("lcampillos/roberta-es-clinical-trials-ner")
df[['first_sentence', 'ner_results']].head(5)

,first_sentence,ner_results
0,In the last 20 years aspecific auto immunity defined as positivity of auto antibodies in blood sample without clinical or biological criteria for defined diseases has been evoked in a subpopulation of infertile women,"[{'entity_group': 'DISO', 'score': 0.7312285, 'word': ' auto', 'start': 31, 'end': 35}, {'entity_group': 'DISO', 'score': 0.5206237, 'word': 'munity', 'start': 38, 'end': 44}, {'entity_group': 'PROC', 'score': 0.9507815, 'word': ' auto antibodies', 'start': 70, 'end': 85}, {'entity_group': 'ANAT', 'score': 0.98456174, 'word': ' blood', 'start': 89, 'end': 94}, {'entity_group': 'ANAT', 'score': 0.38695672, 'word': ' sample', 'start': 95, 'end': 101}, {'entity_group': 'DISO', 'score': 0.99609226, 'word': ' diseases', 'start': 154, 'end': 162}]"
1,The direct pathogenesis of this auto immunity is unknown but therapeutic immunomodulators prescribed on a case by case basis could favor pregnancy even in cases of unexplained primary or secondary infertility,"[{'entity_group': 'CHEM', 'score': 0.99836284, 'word': ' immun', 'start': 73, 'end': 78}, {'entity_group': 'CHEM', 'score': 0.99834275, 'word': 'omod', 'start': 78, 'end': 82}, {'entity_group': 'CHEM', 'score': 0.99750656, 'word': 'ula', 'start': 82, 'end': 85}, {'entity_group': 'CHEM', 'score': 0.99377656, 'word': 'tors', 'start': 85, 'end': 89}, {'entity_group': 'PROC', 'score': 0.7410072, 'word': ' prescri', 'start': 90, 'end': 97}, {'entity_group': 'PROC', 'score': 0.5425408, 'word': 'bed', 'start': 97, 'end': 100}, {'entity_group': 'DISO', 'score': 0.9526619, 'word': ' infer', 'start': 197, 'end': 202}, {'entity_group': 'DISO', 'score': 0.9077292, 'word': 'til', 'start': 202, 'end': 205}, {'entity_group': 'DISO', 'score': 0.95677084, 'word': 'ity', 'start': 205, 'end': 208}]"
2,Meta analysis of studies that evaluated the role of NK cells in IVF outcome showed no significant difference in live birth rates in women with elevated NK cells or NK cell activity compared with women without elevated peripheral NK cells or NK cell activity NK activity assessed using a cytotoxicity assay relative risk 0,"[{'entity_group': 'ANAT', 'score': 0.9837609, 'word': ' NK', 'start': 52, 'end': 54}, {'entity_group': 'ANAT', 'score': 0.5299402, 'word': ' cells', 'start': 55, 'end': 60}, {'entity_group': 'PROC', 'score': 0.95708644, 'word': ' IV', 'start': 64, 'end': 66}, {'entity_group': 'PROC', 'score': 0.92627937, 'word': 'F', 'start': 66, 'end': 67}, {'entity_group': 'ANAT', 'score': 0.9534247, 'word': ' NK', 'start': 152, 'end': 154}, {'entity_group': 'ANAT', 'score': 0.8263227, 'word': ' cells', 'start': 155, 'end': 160}, {'entity_group': 'ANAT', 'score': 0.7314529, 'word': ' perip', 'start': 218, 'end': 223}, {'entity_group': 'ANAT', 'score': 0.8339198, 'word': 'heral', 'start': 223, 'end': 228}, {'entity_group': 'ANAT', 'score': 0.89860743, 'word': ' NK', 'start': 229, 'end': 231}, {'entity_group': 'ANAT', 'score': 0.8465602, 'word': ' cells', 'start': 232, 'end': 237}]"
3,On average 10 of infertile couples have unexplained infertility,"[{'entity_group': 'DISO', 'score': 0.96785283, 'word': ' infer', 'start': 52, 'end': 57}, {'entity_group': 'DISO', 'score': 0.9126259, 'word': 'til', 'start': 57, 'end': 60}, {'entity_group': 'DISO', 'score': 0.8966498, 'word': 'ity', 'start': 60, 'end': 63}]"
4,A systematic review was performed PUBMED using the MESH search terms infertility and auto immunity or reproductive technique or assisted reproduction or in vitro fertilization and auto immunity,"[{'entity_group': 'PROC', 'score': 0.8638412, 'word': ' systematic', 'start': 2, 'end': 12}, {'entity_group': 'PROC', 'score': 0.45689088, 'word': ' review', 'start': 13, 'end': 19}, {'entity_group': 'DISO', 'score': 0.90887654, 'word': ' infer', 'start': 69, 'end': 74}, {'entity_group': 'DISO', 'score': 0.96370745, 'word': 'til', 'start': 74, 'end': 77}, {'entity_group': 'DISO', 'score': 0.93543804, 'word': 'ity', 'start': 77, 'end': 80}, {'entity_group': 'PROC', '

In [97]:
tokenModelAndPipe("StivenLancheros/roberta-base-biomedical-clinical-es-finetuned-ner-CRAFT")
df[['first_sentence', 'ner_results']].head(5)

,first_sentence,ner_results
0,In the last 20 years aspecific auto immunity defined as positivity of auto antibodies in blood sample without clinical or biological criteria for defined diseases has been evoked in a subpopulation of infertile women,"[{'entity_group': 'GENE', 'score': 0.9924138, 'word': ' antibodies', 'start': 75, 'end': 85}]"
1,The direct pathogenesis of this auto immunity is unknown but therapeutic immunomodulators prescribed on a case by case basis could favor pregnancy even in cases of unexplained primary or secondary infertility,[]
2,Meta analysis of studies that evaluated the role of NK cells in IVF outcome showed no significant difference in live birth rates in women with elevated NK cells or NK cell activity compared with women without elevated peripheral NK cells or NK cell activity NK activity assessed using a cytotoxicity assay relative risk 0,"[{'entity_group': 'Cell', 'score': 0.99066085, 'word': ' NK cells', 'start': 52, 'end': 60}, {'entity_group': 'Cell', 'score': 0.9893566, 'word': ' NK cells', 'start': 152, 'end': 160}, {'entity_group': 'Cell', 'score': 0.98944414, 'word': ' NK cell', 'start': 164, 'end': 171}, {'entity_group': 'Cell', 'score': 0.9825586, 'word': ' NK cells', 'start': 229, 'end': 237}, {'entity_group': 'Cell', 'score': 0.98356116, 'word': ' NK cell', 'start': 241, 'end': 248}]"
3,On average 10 of infertile couples have unexplained infertility,[]
4,A systematic review was performed PUBMED using the MESH search terms infertility and auto immunity or reproductive technique or assisted reproduction or in vitro fertilization and auto immunity,[]


In [98]:
tokenModelAndPipe("silpakanneganti/roberta-cpt-medical-ner")
df[['first_sentence', 'ner_results']].head(5)

,first_sentence,ner_results
0,In the last 20 years aspecific auto immunity defined as positivity of auto antibodies in blood sample without clinical or biological criteria for defined diseases has been evoked in a subpopulation of infertile women,"[{'entity_group': 'Health_Problem', 'score': 0.32031363, 'word': ' as', 'start': 21, 'end': 23}, {'entity_group': 'Health_Problem', 'score': 0.33792767, 'word': 'pe', 'start': 23, 'end': 25}, {'entity_group': 'Health_Problem', 'score': 0.4585013, 'word': 'cific', 'start': 25, 'end': 30}, {'entity_group': 'Medical_History', 'score': 0.37908673, 'word': ' auto immunity', 'start': 31, 'end': 44}, {'entity_group': 'Medical_History', 'score': 0.18385516, 'word': ' auto', 'start': 70, 'end': 74}, {'entity_group': 'Procedure_Name', 'score': 0.55657184, 'word': ' antibodies in blood sample', 'start': 75, 'end': 101}]"
1,The direct pathogenesis of this auto immunity is unknown but therapeutic immunomodulators prescribed on a case by case basis could favor pregnancy even in cases of unexplained primary or secondary infertility,"[{'entity_group': 'Health_Problem', 'score': 0.642036, 'word': ' primary or secondary infertility', 'start': 176, 'end': 208}]"
2,Meta analysis of studies that evaluated the role of NK cells in IVF outcome showed no significant difference in live birth rates in women with elevated NK cells or NK cell activity compared with women without elevated peripheral NK cells or NK cell activity NK activity assessed using a cytotoxicity assay relative risk 0,"[{'entity_group': 'Procedure_Name', 'score': 0.5006325, 'word': ' NK cells', 'start': 229, 'end': 237}, {'entity_group': 'Procedure_Name', 'score': 0.6526628, 'word': ' NK cell activity', 'start': 241, 'end': 257}, {'entity_group': 'Procedure_Name', 'score': 0.26796052, 'word': 'ot', 'start': 290, 'end': 292}, {'entity_group': 'Procedure_Name', 'score': 0.6617189, 'word': ' assay', 'start': 300, 'end': 305}]"
3,On average 10 of infertile couples have unexplained infertility,"[{'entity_group': 'Health_Problem', 'score': 0.888003, 'word': ' unexplained infertility', 'start': 40, 'end': 63}]"
4,A systematic review was performed PUBMED using the MESH search terms infertility and auto immunity or reproductive technique or assisted reproduction or in vitro fertilization and auto immunity,"[{'entity_group': 'Health_Problem', 'score': 0.5270905, 'word': ' infertility', 'start': 69, 'end': 80}, {'entity_group': 'Procedure_Name', 'score': 0.37464055, 'word': ' and auto immunity', 'start': 81, 'end': 98}, {'entity_group': 'Procedure_Name', 'score': 0.8535948, 'word': ' reproductive technique or assisted reproduction', 'start': 102, 'end': 149}, {'entity_group': 'Procedure_Name', 'score': 0.9973218, 'word': ' in', 'start': 153, 'end': 155}, {'entity_group': 'Procedure_Name', 'score': 0.79102707, 'word': ' vitro fertilization and auto immunity', 'start': 156, 'end': 193}]"


In [99]:
tokenModelAndPipe("raynardj/ner-disease-ncbi-bionlp-bc5cdr-pubmed")
df[['first_sentence', 'ner_results']].head(5)

,first_sentence,ner_results
0,In the last 20 years aspecific auto immunity defined as positivity of auto antibodies in blood sample without clinical or biological criteria for defined diseases has been evoked in a subpopulation of infertile women,[]
1,The direct pathogenesis of this auto immunity is unknown but therapeutic immunomodulators prescribed on a case by case basis could favor pregnancy even in cases of unexplained primary or secondary infertility,"[{'entity_group': 'Disease', 'score': 0.57288957, 'word': ' infertility', 'start': 197, 'end': 208}]"
2,Meta analysis of studies that evaluated the role of NK cells in IVF outcome showed no significant difference in live birth rates in women with elevated NK cells or NK cell activity compared with women without elevated peripheral NK cells or NK cell activity NK activity assessed using a cytotoxicity assay relative risk 0,"[{'entity_group': 'Disease', 'score': 0.8083944, 'word': ' IVF', 'start': 64, 'end': 67}]"
3,On average 10 of infertile couples have unexplained infertility,"[{'entity_group': 'Disease', 'score': 0.7104395, 'word': ' infertility', 'start': 52, 'end': 63}]"
4,A systematic review was performed PUBMED using the MESH search terms infertility and auto immunity or reproductive technique or assisted reproduction or in vitro fertilization and auto immunity,"[{'entity_group': 'Disease', 'score': 0.5671895, 'word': ' immunity', 'start': 90, 'end': 98}]"
